## Specificity Mapping

This is a bit more complex than sensitivity mapping.

It requires a higher understanding of statistics. I don't want to go into too much depth, so I'll simplify things a lot.

Regression: A way of measuring the strength of the relationship between variables.
T-test: Is the average value

### Statistical function I wrote myself

In [21]:
import numpy as np
from numpy.linalg import pinv

def welchs_t_glm(data, design_matrix, contrast_matrix):
    # Solve the GLM
    XTX = design_matrix.T @ design_matrix
    XTX_inv_pseudo = pinv(XTX)  # Pseudo-inverse
    betas = XTX_inv_pseudo @ design_matrix.T @ data  # Beta coefficients

    # Calculate residuals
    model_predictions = design_matrix @ betas
    residuals = data - model_predictions

    # Contrast values
    contrast_values = contrast_matrix @ betas

    # Variance of the contrast estimate
    var_contrast = contrast_matrix @ XTX_inv_pseudo @ contrast_matrix.T

    # Degrees of freedom using Welch–Satterthwaite equation
    mse = np.sum(residuals**2, axis=0) / (data.shape[0] - design_matrix.shape[1])
    se_contrast = np.sqrt(var_contrast * mse)
    
    # Calculate t-values
    t_values = contrast_values / se_contrast

    return t_values.flatten()


In [23]:
import pandas as pd
from nilearn.maskers import NiftiMasker

df_cases = pd.read_csv('example_data/example_dataset.csv')
df_controls = pd.read_csv('example_data/example_control_dataset.csv')
df_together = pd.concat([df_cases, df_controls], axis=0)
df_together['dataset'] = np.hstack([np.ones(df_cases.shape[0]), np.zeros(df_controls.shape[0])])
df_together['intercept'] = 1

masker = NiftiMasker(mask_img='example_data/MNI152_T1_2mm_brain_mask.nii.gz').fit()

data = masker.transform(df_together['t'])
design_matrix = df_together[['dataset', 'intercept']].values
contrast_matrix = np.array([1, 0])

t_values = welchs_t_glm(data, design_matrix, contrast_matrix)
t_values_img = masker.inverse_transform(t_values)
t_values_img.to_filename('example_data/t_values.nii.gz')


/var/folders/s3/ntpkdn8j7jd5jksmwwldz11h0000gn/T/ipykernel_50641/3640074853.py:25: RuntimeWarning: invalid value encountered in divide
  t_values = contrast_values / se_contrast
